In [ ]:
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

### Helper function

In [ ]:
#init weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [ ]:
#init bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [ ]:
#conv2D
def conv2d(x,w):
    #x --> input tensor -- > [batch,Height,Width,Channels]
    #w --> [filter H, filter W, channels in, channels out]
    return tf.nn.conv2d(x, w, strides=[1,1,1,1], padding='SAME') #'SAME' is 0 padding
    

In [ ]:
#pooling
def max_pooling_2x2(x):
    #x --> input tensor -- > [batch,Height,Width,Channels]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    # ksize = for each image, 2x2 pool along one channel
    # strides = for each image, 2x2 stride along one channel


In [ ]:
    # CONVOLUTIONAL LAYER
    def convolutional_layer(input_x, shape):
        w = init_weights(shape)
        b = init_bias([shape[3]])
        return tf.nn.relu(conv2d(input_x,w)+b)

In [ ]:
# Normal layer
def normal_dense_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    w = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, w) + b

In [ ]:
x = tf.placeholder(tf.float32, shape=[None,784])
y_label = tf.placeholder(tf.float32, shape=[None,10])

In [ ]:
#deflattening the array to make them an image again
x_image = tf.reshape(x, [-1,28,28,1])

In [ ]:
conv1 = convolutional_layer(x_image, shape=[5,5,1,32]) #32 features, 32 filter matrices, 1 is input channel
pool1 = max_pooling_2x2(conv1)

In [ ]:
conv2 = convolutional_layer(pool1, shape=[5,5,32,64]) #64 features with 32 channels
pool2 = max_pooling_2x2(conv2) 

In [ ]:
conv2flat =tf.reshape(pool2,[-1,7*7*64])

In [ ]:
full_layer1 = tf.nn.relu(normal_dense_layer(conv2flat, 1024)) #1024  neurons

In [ ]:
#drop out
hold_prob = tf.placeholder(tf.float32)
full_layer_dropout = tf.nn.dropout(full_layer1, keep_prob=hold_prob)

In [ ]:
y_pred = normal_dense_layer(full_layer_dropout,10) #10 labels

In [ ]:
#loss function
cross_en = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_label, logits=y_pred))

In [ ]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_en)

In [ ]:
init = tf.global_variables_initializer()

In [ ]:
steps = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, feed_dict = {x:batch_x, y_label:batch_y, hold_prob:0.5,})
        if(i%100 == 0):
            print("ON STEP {}".format(i))
            print("Accuracy:")
            matches = tf.equal(tf.arg_max(y_pred,1), tf.arg_max(y_label,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc, feed_dict= {x:mnist.test.images, y_label:mnist.test.labels, hold_prob:1.0}))
            print('\n')